# Комментарии

Колонку ***client_age*** мы решили реализовать выводом полных лет, так как только полная дата может повлиять на ассортимент предлагаемых клиенту товаров, а также помогает определить, с какого дня клиенту можно предоставить скидку в честь дня рождения. Также простое число проще использовать в дальнейнем для категоризации клиентов по возрасту.

Колонку ***client_registration_age*** мы решили реализовать через год и месяц со дня регистрации, так как полных лет регистрации у клиентов не так много. Возможно, магазин решит предоставить клиенту скидку раньше, чем наступит следующий полный год. Также значение поля можно выводить клиенту в приложении (если таковое имеется), чтобы он видел, как долго он пользуется сервисом магазина.

Колонка ***client_category*** реализована через аггрегацию уже совершенных покупок. Мы предполагаем, что таковыми считаются покупки, у которых status = 3 ("оплачено", как нам видится). При этом до тех пор, пока клиент не оплатил хотя бы одну покупку, его категория будет оставаться 'new', и только со следуюшего заказа будет отображаться 'old'.

Последние 2 колонки были приведены в формат Int32, чтобы pandahouse мог их прочитать. 

В задании не уточнялось, нужно ли включать в фильтрацию 1 августа 2020 года. Мы решили включить полные 3 месяца (1 квартал), то есть числа до 31 июля 2020 года включительно.

In [ ]:
answer_3 = '''
SELECT 
    purchase_date, 
    purchase_id, 
    client_id, 
    client_age,
    client_registration_age,
    client_category, 
    promotion_name,
    category_name,
    partner_name,
    client_city, 
    city, 
    toInt32(revenue) as revenue, 
    toInt32(quantity) as quantity
FROM 
    (SELECT *
    FROM
        (SELECT DISTINCT(*) 
        FROM 
            (SELECT * 
            FROM 
                (SELECT city_id, city FROM project_variant_2.city) 
            AS cities_1
            JOIN
                (SELECT *
                FROM    
                    (SELECT 
                        purchase_date, 
                        purchase_id, 
                        client_id,
                        quantity,
                        price * quantity AS revenue,
                        status,
                        city_id,
                        promotion_id
                    FROM project_variant_2.purchase) 
                AS purchase_revenue
                JOIN 
                    (SELECT 
                        client_id, 
                        client_category, 
                        purchase_id 
                    FROM 
                        (SELECT * FROM project_variant_2.purchase) 
                        AS purchases_1
                        JOIN
                            (SELECT 
                                client_id,
                                purchase_id,
                                completed_purchases,
                                first_comleted_purchase,
                                purchase_date,
                                status,
                                CASE 
                                    WHEN if_completed = 'yes' and first_comleted_purchase < purchase_date 
                                    THEN 'old'
                                    ELSE 'new'
                                END AS client_category
                            FROM 
                                (SELECT * FROM project_variant_2.purchase) AS is_completed
                                LEFT JOIN
                                    (SELECT 
                                        client_id,
                                        Count(DISTINCT(purchase_id)) AS completed_purchases,
                                        min(purchase_date) as first_comleted_purchase,
                                        CASE 
                                            WHEN completed_purchases >= 1 
                                            THEN 'yes'
                                        END AS if_completed
                                    FROM project_variant_2.purchase
                                    WHERE status = 3 
                                    GROUP BY client_id) AS counter
                                ON is_completed.client_id = counter.client_id)
                        AS categories
                        ON purchases_1.purchase_id = categories.purchase_id)
                    AS purchase_category
                    ON 
                    purchase_revenue.purchase_id = purchase_category.purchase_id)                     
            AS agg_purchases
            ON cities_1.city_id = agg_purchases.city_id) 
            AS purchase_city
            JOIN 
                (SELECT * 
                FROM project_variant_2.promotion) AS promotions
            ON promotions.promotion_id = purchase_city.promotion_id) 
    AS l
    JOIN
        (SELECT *
        FROM 
            (SELECT DISTINCT(*) 
            FROM
                (SELECT 
                    client_city, 
                    client_city_id 
                FROM project_variant_2.city) AS cities_2
                JOIN 
                    (SELECT 
                        client_id, 
                        client_city_id, 
                        CASE
                            WHEN 
                                toDayOfMonth(today()) >= toDayOfMonth(registration)
                            THEN 
                                CONCAT(toString(intDiv(dateDiff(month, registration, today()) + 1, 12)), ' y ',
                                toString(modulo(dateDiff(month, registration, today()) + 1, 12)), ' m')
                            ELSE 
                                CONCAT(toString(intDiv(dateDiff(month, registration, today()), 12)), ' y ',
                                toString(modulo(dateDiff(month, registration, today()), 12)), ' m')
                        END AS client_registration_age,
                        CASE
                            WHEN 
                                toDayOfYear(today()) >= toDayOfYear(birth_date)
                            THEN 
                                dateDiff(year, birth_date, today())
                            ELSE 
                                dateDiff(year, birth_date, today()) - 1 
                        END AS client_age
                    FROM project_variant_2.client) AS clients
                ON cities_2.client_city_id = clients.client_city_id)) 
        AS r
        ON l.client_id = r.client_id)
HAVING 
    status = 1
    AND 
    purchase_date BETWEEN '2020-05-01' and '2020-07-31'
ORDER BY purchase_date
    '''